<a href="https://colab.research.google.com/github/wozzin/AI_deeplearning/blob/main/abalone_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def main_execute(epoch_count = 10, mb_size = 2, report = 2, train_ratio = 0.8):
    load_dataset()
    weight_initial, bias_initial = init_param()
    losses_mean_row, accs_mean_row, final_acc = train_and_test(epoch_count,
                                                               mb_size, 
                                                               report, 
                                                               train_ratio)

    return weight_initial, bias_initial, losses_mean_row, accs_mean_row, final_acc

In [ ]:
def load_dataset():
    with open('/content/abalone.csv') as csvfile:
    #with open('/content/abalone_mini.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt 
    
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'M' : data[n, 0] = 1
        if row[0] == 'F' : data[n, 1] = 1
        if row[0] == 'I' : data[n, 2] = 1
        data[n, 3 : ]= row[1:]

In [ ]:
%run /content/MathUtils.ipynb

In [ ]:
def init_param():
    global weight, bias 

    weight_initial = []
    bias_initial   = []
    weight = np.random.normal(RND_MEAN, RND_STD, size = [input_cnt, output_cnt])
    bias   = np.zeros([output_cnt])
    print("Initial Weight Value : \n{}".format(weight))
    print("Initial Bias Value : \n{}".format(bias))
    weight_initial.append(weight)
    bias_initial.append(bias)

    return weight_initial, bias_initial

In [ ]:
def arrange_data(mb_size, train_ratio):
    
    global shuffle_map, test_begin_index

    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)

    mini_batch_step_count = int(data.shape[0] * train_ratio) // mb_size
    test_begin_index = mini_batch_step_count * mb_size
    return mini_batch_step_count

In [ ]:
def get_test_data():
    test_data = data[shuffle_map[test_begin_index:]]
    return test_data[ : , : -output_cnt], test_data[ : , -output_cnt : ]

In [ ]:
def get_train_data(mb_size, n):
    if n == 0:
        np.random.shuffle(shuffle_map[:test_begin_index])

    train_data = data[shuffle_map[mb_size * n : mb_size * (n+1) ]]

    return train_data[ : , : -output_cnt], train_data[ : , -output_cnt : ]

In [ ]:
def train_and_test(epoch_count, mb_size, report, train_ratio):
    mini_batch_step_count = arrange_data(mb_size, train_ratio)
    test_x, test_y = get_test_data()

    losses_mean_row = []
    accs_mean_row   = []

    for epoch in range(epoch_count):
        losses = []
        accs   = [] 
        for n in range(mini_batch_step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print("Epoch {} : Train - Loss = {:.3f}, Accuracy = {:.3f}  / Test - Accuracy = {:.3f}".\
                  format(epoch + 1, np.mean(losses), np.mean(accs), acc))
            
        losses_mean = np.mean(losses)
        accs_mean   = np.mean(accs) * 100

        losses_mean_row.append(losses_mean)
        accs_mean_row.append(accs_mean)

    final_acc = run_test(test_x, test_y)        
    print("=" * 30, "Final TEST", "=" * 30)
    print("\nFinal Accuracy : {:.3f}".format(final_acc))

    return losses_mean_row, accs_mean_row, final_acc


In [ ]:
def forward_neuralnet(x):
    y_hat = np.matmul(x, weight) + bias 
    return y_hat, x

In [ ]:
def forward_postproc(y_hat, y):
    diff   = y_hat - y
    square = np.square(diff)
    loss   = np.mean(square)

    return loss, diff

In [ ]:
def eval_accuracy(y_hat, y):
    mdiff = np.mean(np.abs((y_hat - y) / y))
    return 1 - mdiff 

In [ ]:
def backprop_neuralnet(G_output, x):
    global weight, bias 

    x_transpose = x.transpose()
    G_w = np.matmul(x_transpose, G_output)

    G_b = np.sum(G_output, axis = 0)

    weight -= LEARNING_RATE * G_w
    bias   -= LEARNING_RATE * G_b

In [ ]:
def backprop_postproc(diff):
    M_N = diff.shape

    g_mse_square  = np.ones(M_N) / np.prod(M_N)
    g_square_diff = 2 * diff 
    g_diff_output = 1

    G_diff   = g_mse_square * g_square_diff 
    G_output = g_diff_output * G_diff

    return G_output 

In [ ]:
def run_train(x, y):
    y_hat, aux_nn_x   = forward_neuralnet(x)
    loss, aux_pp_diff = forward_postproc(y_hat, y)

    accuracy = eval_accuracy(y_hat, y)

    G_output = backprop_postproc(aux_pp_diff)
    backprop_neuralnet(G_output, aux_nn_x)

    return loss, accuracy 

In [ ]:
def run_test(x,y):
    y_hat, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(y_hat, y)

    return accuracy